### 필요한 패키지 임포트 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/20231212_상세페이지용_중분류컬럼_추가.xlsx')

In [ ]:
data=data[data['휴폐업여부']!='Y']

#### 1. 가중치 선정
###### 매출액에 영향력을 크게 주는 지표일 수록 가중치 높게 선정됨
###### 방법: 각 지표의 클래스 별 매출액과의 상관관계 분석을 통해 가중치 산정
###### 지표: 수출여부, 업종별 수출액 비중, 3년 평균 매출액, 매출액대비연구개발비, 영업이익율, 기업규모, 특허유무, 부채비율

In [ ]:
# 매출액 영향력을 크게 줘야하므로, 매출액이 없는 행은 삭제한 후 상관관계 분석을 위한 데이터를 구축함
data=data.dropna(subset=['매출액_2022'])

In [ ]:
len(data)

### 1 각 지표별 상관분석

#### 1-1 영업이익율

##### 1-1-1 영업이익율을 기준으로 기업들을 1-10점으로 구간화
#####   -> 각 점수별 기업수와 총매출액을 구함



In [ ]:
#영업이익율
filtered_df=data.copy()
filtered_df=filtered_df[filtered_df['매출액_2022']!=0]

#영업이익율 컬럼 생성
filtered_df['영업이익율']=filtered_df['영업이익_2022']/filtered_df['매출액_2022']

In [ ]:
#영업이익율 10 class로 구간화
quantiles_7 = filtered_df['영업이익율'].quantile(np.linspace(0, 1, 11)[1:-1]).values

grade_labels_7 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
filtered_df['영업이익율_등급'] = pd.cut(filtered_df['영업이익율'], bins=np.concatenate([[-np.inf], quantiles_7, [np.inf]]), labels=grade_labels_7)

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df.groupby('영업이익율_등급')['영업이익율'].count()
grade_total_revenue = filtered_df.groupby('영업이익율_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_영업이익율 = pd.DataFrame({
    '등급': grade_labels_7,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-1-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_영업이익율['기업수'].sum()
total_revenue = df_영업이익율['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_영업이익율['value'] = (df_영업이익율['매출액총합']/total_revenue) / (df_영업이익율['기업수']/total_company_count)
df_영업이익율['value'] = df_영업이익율['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_영업이익율 = df_영업이익율['매출액총합'].corr(df_영업이익율['value'])
corr_영업이익율

#### 1-2 3년매출액평균

In [ ]:
#매출액 3년평균
filtered_df1=data.copy()

In [ ]:
#매출액 평균을 구하기 위한 함수
#2020,2021,2022년 중 특정 년도의 데이터가 없을 수 있으므로 해당 함수를 통해 있는 년도만으로 평균 계산 진행
def sell_cal(row):
    valid_sales = [sale for sale in [row['매출액_2022'], row['매출액_2021'], row['매출액_2020']] if pd.notna(sale)]
    return sum(valid_sales) / len(valid_sales)

In [ ]:
#매출액평균 컬럼 생성
filtered_df1['매출액평균'] = filtered_df1.apply(sell_cal, axis=1)

###### 1-2-1 매출액평균을 기준으로 기업들을 1-10점으로 구간화
######   -> 각 점수별 기업수와 총매출액을 구함

In [ ]:
#매출액평균 10 class로 구간화
quantiles_7 = filtered_df1['매출액평균'].quantile(np.linspace(0, 1, 11)[1:-1]).values

grade_labels_7 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
filtered_df1['매출액평균_등급'] = pd.cut(filtered_df1['매출액평균'], bins=np.concatenate([[-np.inf], quantiles_7, [np.inf]]), labels=grade_labels_7)

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df1.groupby('매출액평균_등급')['매출액평균'].count()
grade_total_revenue = filtered_df1.groupby('매출액평균_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_매출액평균 = pd.DataFrame({
    '등급': grade_labels_7,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-2-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_매출액평균['기업수'].sum()
total_revenue = df_매출액평균['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_매출액평균['value'] = (df_매출액평균['매출액총합']/total_revenue) / (df_영업이익율['기업수']/total_company_count)
df_매출액평균['value'] = df_매출액평균['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_매출액평균 = df_매출액평균['매출액총합'].corr(df_매출액평균['value'])
corr_매출액평균

#### 1-3 매출액 대비 연구개발비

In [ ]:
#매출액이 0이면 제외
filtered_df2=data.copy()
filtered_df2=filtered_df2.dropna(subset=['매출액_2022'])
filtered_df2=filtered_df2[filtered_df2['매출액_2022']!=0]
#연구개발비가 음수이면 0으로 대체
filtered_df2.loc[filtered_df2['연구개발비_2022'] < 0, '연구개발비_2022'] = 0

In [ ]:
#매출액대비연구개발비 컬럼 만들기
filtered_df2['매출액대비연구개발비']= filtered_df2['연구개발비_2022']/filtered_df2['매출액_2022']

##### 1-3-1 매출액 대비 연구개발비를 기준으로 기업들을 1-10점으로 구간화
#####   -> 각 점수별 기업수와 총매출액을 구함

In [ ]:
#매출액대비 연구개발비 10 class로 구간화
#0을 하나의 구간으로 갖고, 나머지 값을 9개의 구간으로 나눔
bin_0 = [-np.inf, 0]
non_zero_values = filtered_df2.loc[filtered_df2['매출액대비연구개발비'] > 0, '매출액대비연구개발비']
quantiles_9 = non_zero_values.quantile(np.linspace(0, 1, 10)[1:-1]).tolist()
bins = bin_0 + quantiles_9 + [np.inf]

grade_labels_10 = [1,2,3,4,5,6,7,8,9,10]
filtered_df2['매출액대비연구개발비_등급'] = pd.cut(filtered_df2['매출액대비연구개발비'], bins=bins, labels=grade_labels_10, include_lowest=True)

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df2['매출액대비연구개발비_등급'].value_counts().sort_index()
grade_total_revenue = filtered_df2.groupby('매출액대비연구개발비_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_매출액대비연구개발비 = pd.DataFrame({
    '등급': grade_labels_10,
    '기업수': grade_counts.reindex(grade_labels_10).fillna(0).astype(int).values,  # 누락된 구간을 0으로 채웁니다.
    '매출액총합': grade_total_revenue.reindex(grade_labels_10).fillna(0).values
})

###### 1-3-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_매출액대비연구개발비['기업수'].sum()
total_revenue = df_매출액대비연구개발비['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_매출액대비연구개발비['value'] = (df_매출액대비연구개발비['매출액총합']/total_revenue) / (df_매출액대비연구개발비['기업수']/total_company_count)
df_매출액대비연구개발비['value'] = df_매출액대비연구개발비['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_매출액대비연구개발비 = df_매출액대비연구개발비['매출액총합'].corr(df_매출액대비연구개발비['value'])
corr_매출액대비연구개발비

#### 1-4 특허,이노비즈,메인비즈 유무

In [ ]:
#특허,이노,메인 유무
filtered_df3 = data.copy()

In [ ]:
#특허등록건수,이노비즈여부,메인비즈여부 컬럼을 이용하여 3개 모두 있으면 10 값을 가지고 2개만 있으면 7점, 1개만 있으면 5점 0개면 1 값을 가지게 됨
def assign_patent(row):
  if (row['특허등록건수']!=0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='유'):
    return 10
  elif ((row['특허등록건수']!=0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='무')):
    return 7
  elif ((row['특허등록건수']!=0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='유')):
    return 7
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='유')):
    return 7
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='무')):
    return 5
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='유')):
    return 5
  elif ((row['특허등록건수']!=0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='무')):
    return 5
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='무')):
    return 1

In [ ]:
#위에서 생성한 함수를 이용하여 특허등록유무 컬럼 생성
filtered_df3['특허이노메인_등급'] = filtered_df3.apply(assign_patent, axis=1)

##### 1-4-1특허등록건수,이노비즈여부,메인비즈여부 컬럼을 이용하여 3개 모두 있으면 10 값을 가지고 2개만 있으면 7점, 1개만 있으면 5점 0개면 1점 부여
#####   -> 각 점수별 기업수와 총매출액을 구한 후 상관분석 진행

In [ ]:
#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df3['특허이노메인_등급'].value_counts().sort_index()
grade_total_revenue = filtered_df3.groupby('특허이노메인_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_특허이노메인유무 = pd.DataFrame({
    '등급': grade_counts.index,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-4-2 특허이노메인 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_특허이노메인유무['기업수'].sum()
total_revenue = df_특허이노메인유무['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_특허이노메인유무['value'] = (df_특허이노메인유무['매출액총합']/total_revenue) / (df_특허이노메인유무['기업수']/total_company_count)
df_특허이노메인유무['value'] = df_특허이노메인유무['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_특허이노메인유무 = df_특허이노메인유무['매출액총합'].corr(df_특허이노메인유무['value'])
corr_특허이노메인유무

#### 1-5 기업규모

In [ ]:
#기업규모
filtered_df4=data.copy()
filtered_df4=filtered_df4.dropna(subset=['기업규모'])
filtered_df4=filtered_df4[filtered_df4['기업규모']!='판단제외']

In [ ]:
# 기업규모 컬럼을 이용하여 소기업,소상공인,중기업,한시성중소기업은 1값을 가지고
#중견기업, 보호대상중견기업 2, 대기업은 1 값을 가지는 함수 구현
def assign_size_status(row):
    if row['기업규모'] == '대기업':
        return 3
    elif row['기업규모'] == '중견기업' or row['기업규모']=='보호대상중견기업':
        return 2
    else:
      return 1

In [ ]:
# 기업규모_수정 컬럼 생성
filtered_df4['기업규모_수정'] = filtered_df4.apply(assign_size_status, axis=1)

##### 1-5-1 기업규모 기준으로 중소기업 10점, 중견기업 5점, 대기업 1점 부여
#####   -> 각 점수별 기업수와 총매출액을 구함

In [ ]:
#기업규모
#1,4,7
#중소기업->10 중견기업->5 대기업->1
filtered_df4['기업규모_등급'] = filtered_df4['기업규모_수정'].map({1:10, 2:5, 3:1})

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df4['기업규모_등급'].value_counts().sort_index()
grade_total_revenue = filtered_df4.groupby('기업규모_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_기업규모 = pd.DataFrame({
    '등급': grade_counts.index,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-5-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_기업규모['기업수'].sum()
total_revenue = df_기업규모['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_기업규모['value'] = (df_기업규모['매출액총합']/total_revenue) / (df_기업규모['기업수']/total_company_count)
df_기업규모['value'] = df_기업규모['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_기업규모 = df_기업규모['매출액총합'].corr(df_기업규모['value'])
corr_기업규모

#### 1-6 부채비율

In [ ]:
#부채비율이 자본잠식인 값을 제거
filtered_df6 = data.copy()
filtered_df6=filtered_df6.dropna(subset=['부채비율_2022'])
filtered_df6 = filtered_df6[filtered_df6['부채비율_2022'] != '자본잠식']

In [ ]:
#부채비율컬럼 타입 변경
filtered_df6['부채비율_2022']=filtered_df6['부채비율_2022'].astype(float)

In [ ]:
# 부채비율_2022컬럼을 기준으로 200%이하는 3, 200-400%는 2, 400%이상은 1값을 가지는 함수 구현
def assign_status(row):
    if row['부채비율_2022']<200:
        return 3
    elif row['부채비율_2022'] >=200 and row['부채비율_2022']<400:
        return 2
    else:
      return 1

In [ ]:
#위의 함수를 적용한 부채비율구간 컬럼 생성
filtered_df6['부채비율구간'] = filtered_df6.apply(assign_status, axis=1)

##### 1-6-1 부채비율을 기준으로 기업들을 200%미만은 10점, 200-400%는 5점, 400%이상은 1점 부여
#####   -> 각 점수별 기업수와 총매출액을 구함

In [ ]:
#부채비율구간
filtered_df6['부채비율_등급'] = filtered_df6['부채비율구간'].map({3:10, 2:5, 1:1})

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df6['부채비율_등급'].value_counts().sort_index()
grade_total_revenue = filtered_df6.groupby('부채비율_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_부채비율구간 = pd.DataFrame({
    '등급': grade_counts.index,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-6-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_부채비율구간['기업수'].sum()
total_revenue = df_부채비율구간['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_부채비율구간['value'] = (df_부채비율구간['매출액총합']/total_revenue) / (df_부채비율구간['기업수']/total_company_count)
df_부채비율구간['value'] = df_부채비율구간['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_부채비율구간 = df_부채비율구간['매출액총합'].corr(df_부채비율구간['value'])
corr_부채비율구간

#### 1-7 수출여부

In [ ]:
filtered_df5=data.copy()

In [ ]:
# EU,US둘다 수출하면 4, EU만 수출하면 3, US만 수출하면 2, 이외 국가에 수출하면 1,
# 수출기업이 아니면 0 값이 되는 함수 구현
def assign_export_status(row):
    if row['수출 여부 (EU)'] == '유' and row['수출 여부 (US)'] == '유':
      return 4
    elif row['수출 여부 (EU)'] == '유':
      return 3
    elif row['수출 여부 (US)'] == '유':
      return 2
    elif pd.notna(row['수출액_2022']):
      return 1
    else:
      return 0

In [ ]:
#위의 함수를 적용한 수출여부 컬럼 생성
filtered_df5['수출여부'] = filtered_df5.apply(assign_export_status, axis=1)

##### 1-7-1 수출여부를 기준으로 기업들을 EU,US 둘다 수출하면 10점, EU만 수출하면 9점, US만 수출하면 8점, 이외 국가에 수출하면 5점, 수출 안하는 기업은 1점 부여
#####   -> 각 점수별 기업수와 총매출액을 구함

In [ ]:
#수출여부
filtered_df5['수출여부_등급'] = filtered_df5['수출여부'].map({4:10, 3:9, 2:8, 1:5, 0:1})

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df5['수출여부_등급'].value_counts().sort_index()
grade_total_revenue = filtered_df5.groupby('수출여부_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_수출여부 = pd.DataFrame({
    '등급': grade_counts.index,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-7-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_수출여부['기업수'].sum()
total_revenue = df_수출여부['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_수출여부['value'] = (df_수출여부['매출액총합']/total_revenue) / (df_수출여부['기업수']/total_company_count)
df_수출여부['value'] = df_수출여부['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_수출여부 = df_수출여부['매출액총합'].corr(df_수출여부['value'])
corr_수출여부

#### 1-8 업종별 수출액 비중

In [ ]:
filtered_df7 = data.copy()
filtered_df7=filtered_df7.dropna(subset=['중분류'])

##### 1-8-1 업종별 수출 비중을 기준으로 점수 부여
#####   >> 석유화학 10점, 철강: 9점, 전기전자/기계: 8점, 운송장비: 7점, 음식료/정보통신: 6점, 섬유의복: 5점, 기타제조; 4점, 목재종이: 3점, 비금속: 2점, 출판업/기타비제조: 1점
#####   -> 각 점수별 기업수와 총매출액을 구한 후 상관분석 진행

In [ ]:
filtered_df7['업종별수출비중_등급'] = filtered_df7['중분류'].map({'석유화학':10,
                                                      '철강':9,'전기전자':8,'기계':8,'운송장비':7,
                                                      '음식료':6,'정보통신':6,
                                                      '섬유의복':5,'기타제조':4,'목재종이':3,'비금속':2, '출판업':1, '기타비제조':1})

#각 class별로 기업수와 매출액 총합 계산
grade_counts = filtered_df7['업종별수출비중_등급'].value_counts().sort_index()
grade_total_revenue = filtered_df7.groupby('업종별수출비중_등급')['매출액_2022'].sum()

#각 class별 기업수와 매출액총합 정보가 있는 데이터프레임 생성
df_업종별수출비중 = pd.DataFrame({
    '등급': grade_counts.index,
    '기업수': grade_counts.values,
    '매출액총합': grade_total_revenue.values
})

##### 1-8-2 매출액에 대한 impact계산과 이를 이용한 상관관계 분석

In [ ]:
#전체 기업수와 전체 매출액총합을 구함
total_company_count = df_업종별수출비중['기업수'].sum()
total_revenue = df_업종별수출비중['매출액총합'].sum()

# 전체값을 구한 것을 각 class별 값에 나눠줌
df_업종별수출비중['value'] = (df_업종별수출비중['매출액총합']/total_revenue) / (df_업종별수출비중['기업수']/total_company_count)
df_업종별수출비중['value'] = df_업종별수출비중['value'].round(3)

In [ ]:
#영향값과 매출액간의 상관관계 분석
corr_업종별수출비중 = df_업종별수출비중['매출액총합'].corr(df_업종별수출비중['value'])
corr_업종별수출비중

### 2 각 지표별 가중치 계산

##### 계산 방식 설명
##### 1. 각 지표 당 10점까지 부여되었으므로, 각 지표의 상관계수마다 10을 곱해줌
##### 2. 각 지표별로 10을 곱해준 값을 총합함
##### 3. 각 지표의 상관계수에 총합해준 값을 나눠줌 -> 해당 값이 해당 지표의 가중치로 설정됨

In [ ]:
#1인당매출액
a=corr_영업이익율*10
#매출액대비연구개발비
b=corr_매출액대비연구개발비*10
#부채비율구간
c=corr_부채비율구간*10
#기업규모
d=corr_기업규모*10
#특허등록건수
e=corr_특허이노메인유무*10
#수출국가
f=corr_수출여부*10
#업종별ESG리스크
g=corr_업종별수출비중*10
#매출액평균
h=corr_매출액평균*10


#총합
total = a+b+c+d+e+f+g+h

In [ ]:
#각 지표별 가중치
print('영업이익율 :',(a/total),'매출액대비연구개발비 :',(b/total),
 '부채비율구간 :', (c/total), '기업규모 :', (d/total),'특허이노메인유무: ', (e/total),'\n'
  '수출국가 :', (f/total), '업종별수출비중 :', (g/total), '매출액평균 : ',(h/total))

### 2. 가중치 적용 및 등급화


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/20231212_상세페이지용_중분류컬럼_추가.xlsx')

In [ ]:
df=df[df['휴폐업여부']!='Y']

In [ ]:
dt = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ESG 참여기업 61곳.xlsx')

In [ ]:
df.rename(columns={'사업자등록번호':'사업자번호'}, inplace=True)

In [ ]:
df = pd.merge(df,dt,how='left',on='사업자번호')

#### 2-1 각 지표별 컬럼 생성
###### 방식 설명
###### 1.  <1. 가중치 설정>에서 사용되었던 각 지표별 1-10점 부여 방식을 그대로 사용하여 컬럼 생성 및 점수 부여
###### 2. 기존 데이터와 merge시켜 기존 데이터에 해당 지표의 등급컬럼 추가

##### 2-1-1 영업이익율

In [ ]:
#영업이익율
filtered_df=df.copy()
filtered_df=filtered_df.dropna(subset=['매출액_2022'])
filtered_df=filtered_df[filtered_df['매출액_2022']!=0]

#영업이익율 컬럼 생성
filtered_df['영업이익율']=filtered_df['영업이익_2022']/filtered_df['매출액_2022']

In [ ]:
#영업이익율
quantiles_7 = filtered_df['영업이익율'].quantile(np.linspace(0, 1, 11)[1:-1]).values

grade_labels_7 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
filtered_df['영업이익율_등급'] = pd.cut(filtered_df['영업이익율'], bins=np.concatenate([[-np.inf], quantiles_7, [np.inf]]), labels=grade_labels_7)

In [ ]:
#기존 data와 merge시키기
filtered_df = filtered_df[['사업자번호','영업이익율','영업이익율_등급']]
df = pd.merge(df,filtered_df,how='left',on='사업자번호')

##### 2-1-2  3년 평균 매출액

In [ ]:
#매출액 3년평균
filtered_df1=df.copy()
filtered_df1 = filtered_df1.dropna(subset=['매출액_2022', '매출액_2021', '매출액_2020'], how='all')

In [ ]:
def sell_cal(row):
    valid_sales = [sale for sale in [row['매출액_2022'], row['매출액_2021'], row['매출액_2020']] if pd.notna(sale)]
    return sum(valid_sales) / len(valid_sales)

In [ ]:
#위의 함수를 사용하여 매출액평균 컬럼 생성
filtered_df1['매출액평균'] = filtered_df1.apply(sell_cal, axis=1)

In [ ]:
quantiles_7 = filtered_df1['매출액평균'].quantile(np.linspace(0, 1, 11)[1:-1]).values

grade_labels_7 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
filtered_df1['매출액평균_등급'] = pd.cut(filtered_df1['매출액평균'], bins=np.concatenate([[-np.inf], quantiles_7, [np.inf]]), labels=grade_labels_7)

In [ ]:
#기존 data와 merge시키기
filtered_df1 = filtered_df1[['사업자번호','매출액평균','매출액평균_등급']]
df=pd.merge(df,filtered_df1,how='left',on='사업자번호')

##### 2-1-3 매출액 대비 연구개발비

In [ ]:
#매출액이 0이면 제외
filtered_df2=df.copy()
filtered_df2=filtered_df2.dropna(subset=['매출액_2022'])
filtered_df2=filtered_df2[filtered_df2['매출액_2022']!=0]
#연구개발비가 음수이면 0으로 대체
filtered_df2.loc[filtered_df2['연구개발비_2022'] < 0, '연구개발비_2022'] = 0

In [ ]:
#매출액대비연구개발비 컬럼 생성
filtered_df2['매출액대비연구개발비']= filtered_df2['연구개발비_2022']/filtered_df2['매출액_2022']

In [ ]:
#0을 하나의 구간으로 갖고, 나머지 값을 9개의 구간으로 나눔
bin_0 = [-np.inf, 0]
non_zero_values = filtered_df2.loc[filtered_df2['매출액대비연구개발비'] > 0, '매출액대비연구개발비']
quantiles_9 = non_zero_values.quantile(np.linspace(0, 1, 10)[1:-1]).tolist()
bins = bin_0 + quantiles_9 + [np.inf]

grade_labels_10 = [1,2,3,4,5,6,7,8,9,10]
filtered_df2['매출액대비연구개발비_등급'] = pd.cut(filtered_df2['매출액대비연구개발비'], bins=bins, labels=grade_labels_10, include_lowest=True)

In [ ]:
#기존 data와 merge시키기
filtered_df2 = filtered_df2[['사업자번호','매출액대비연구개발비','매출액대비연구개발비_등급']]
df=pd.merge(df,filtered_df2,how='left',on='사업자번호')

##### 2-1-4 특허이노메인

In [ ]:
filtered_df3 = df.copy()

In [ ]:
#특허등록건수,이노비즈여부,메인비즈여부 컬럼을 이용하여 3개 모두 있으면 10 값을 가지고 2개만 있으면 7점, 1개만 있으면 5점 0개면 1 값을 가지게 되는 함수
def assign_patent(row):
  if (row['특허등록건수']!=0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='유'):
    return 10
  elif ((row['특허등록건수']!=0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='무')):
    return 8
  elif ((row['특허등록건수']!=0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='유')):
    return 8
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='유')):
    return 8
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='유' and row['이노비즈여부']=='무')):
    return 8
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='유')):
    return 8
  elif ((row['특허등록건수']!=0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='무')):
    return 6
  elif ((row['특허등록건수']==0 and row['메인비즈여부']=='무' and row['이노비즈여부']=='무')):
    return 1

In [ ]:
#위의 함수를 사용하여 특허이노메인 컬럼 생성
filtered_df3['특허이노메인_등급'] = filtered_df3.apply(assign_patent, axis=1)

In [ ]:
#기존 data와 merge시키기
filtered_df3 = filtered_df3[['사업자번호','특허이노메인_등급']]
df = pd.merge(df,filtered_df3,how='left',on='사업자번호')

##### 2-1-5 기업규모

In [ ]:
filtered_df4=df.copy()
filtered_df4=filtered_df4.dropna(subset=['기업규모'])
filtered_df4=filtered_df4[filtered_df4['기업규모']!='판단제외']

In [ ]:
# 기업규모
#소기업,소상공인,중기업,한시성중소기업
#중견기업, 보호대상중견기업
#대기업
def assign_size_status(row):
    if row['기업규모'] == '대기업':
        return 3
    elif row['기업규모'] == '중견기업' or row['기업규모']=='보호대상중견기업':
        return 2
    else:
      return 1

In [ ]:
#위의 함수를 사용하여 기업규모_수정 컬럼 생성
filtered_df4['기업규모_수정'] = filtered_df4.apply(assign_size_status, axis=1)

In [ ]:
#기업규모_수정 컬럼을 사용하여 기업규모_등급 컬럼 생성
filtered_df4['기업규모_등급'] = filtered_df4['기업규모_수정'].map({1:10, 2:5, 3:1})

In [ ]:
#기존 data와 merge시키기
filtered_df4 = filtered_df4[['사업자번호','기업규모_수정','기업규모_등급']]
df = pd.merge(df,filtered_df4,how='left',on='사업자번호')

##### 2-1-6 부채비율

In [ ]:
filtered_df6 = df.copy()
filtered_df6=filtered_df6.dropna(subset=['부채비율_2022'])
filtered_df6 = filtered_df6[filtered_df6['부채비율_2022'] != '자본잠식']

In [ ]:
#부채비율컬럼 타입 변경
filtered_df6['부채비율_2022']=filtered_df6['부채비율_2022'].astype(float)

In [ ]:
# 부채비율
def assign_status(row):
    if row['부채비율_2022']<200:
        return 3
    elif row['부채비율_2022'] >=200 and row['부채비율_2022']<400:
        return 2
    else:
      return 1

In [ ]:
#위 함수 적용하여 부채비율구간 컬럼 생성
filtered_df6['부채비율구간'] = filtered_df6.apply(assign_status, axis=1)

In [ ]:
#부채비율구간 컬럼을 이융하여 부채비율_등급 컬럼 생성
filtered_df6['부채비율_등급'] = filtered_df6['부채비율구간'].map({3:10, 2:5, 1:1})

In [ ]:
#기존 data와 merge시키기
filtered_df6 = filtered_df6[['사업자번호','부채비율구간','부채비율_등급']]
df = pd.merge(df,filtered_df6,how='left',on='사업자번호')

##### 2-1-7 수출여부

In [ ]:
filtered_df5=df.copy()

In [ ]:
#수출여부 컬럼 만들기
def assign_export_status(row):
    if row['수출 여부 (EU)'] == '유' and row['수출 여부 (US)'] == '유':
      return 4
    elif row['수출 여부 (EU)'] == '유':
      return 3
    elif row['수출 여부 (US)'] == '유':
      return 2
    elif pd.notna(row['수출액_2022']):
      return 1
    else:
      return 0

In [ ]:
#위의 함수를 사용하여 수출여부 컬럼 생성
filtered_df5['수출여부'] = filtered_df5.apply(assign_export_status, axis=1)

In [ ]:
#수출여부 컬럼을 사용하여 수출여부_등급 컬럼 생성
filtered_df5['수출여부_등급'] = filtered_df5['수출여부'].map({4:10, 3:9, 2:8, 1:5, 0:1})

In [ ]:
#코데이터에서 제공받은 데이터에서 누락된 기업의 수출여부를 채우기 위한 변경

#밑에 변경해준 총 15개의 기업은 산업단지공단에서 ESG 컨설팅 사업을 지원해준 기업임
#밑에 변경해준 기업들은 코데이터에서 제공받은 데이터 상으로 했을 때, 수출여부_등급 컬럼에서 1값이 나옴(수출여부(EU)와수출여부(US)의 value가 '무'이며 수출액_2022은 null값임)
#하지만 산단공측에서 제공받은 지원사업 참여기업 데이터를 확인해 보았을때, 해당 15개 기업은 수출기업이라고 나와있음
#따라서 15개 기업의 수출여부_등급 컬럼의 값을 5(이외 국가 수출의 값)로 변경해줌

filtered_df5.loc[(filtered_df5['사업자번호'] == 1078180508), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 1378107186), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 1398125536), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 3148100664), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 6108136319), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 1298156556), '수출여부_등급']=5

filtered_df5.loc[(filtered_df5['사업자번호'] == 1318607975), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 5030449650), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 5138152551), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 2148647888), '수출여부_등급']=5

filtered_df5.loc[(filtered_df5['사업자번호'] == 5038192857), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 5148128277), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 5038601273), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 5038602608), '수출여부_등급']=5
filtered_df5.loc[(filtered_df5['사업자번호'] == 1088159497), '수출여부_등급']=5

In [ ]:
#기존 data와 merge시키기
filtered_df5 = filtered_df5[['사업자번호','수출여부','수출여부_등급']]
df = pd.merge(df,filtered_df5,how='left',on='사업자번호')

##### 2-1-8 업종별 수출액 비중

In [ ]:
filtered_df7 = df.copy()
filtered_df7=filtered_df7.dropna(subset=['중분류'])

In [ ]:
#업종별수출비중_등급 컬럼 생성
filtered_df7['업종별수출비중_등급'] = filtered_df7['중분류'].map({'석유화학':10,
                                                      '철강':9,'전기전자':8,'기계':8,'운송장비':7,
                                                      '음식료':6,'정보통신':6,
                                                      '섬유의복':5,'기타제조':4,'목재종이':3,'비금속':2, '출판업':1, '기타비제조':1})

In [ ]:
#기존 data와 merge시키기
filtered_df7 = filtered_df7[['사업자번호','업종별수출비중_등급']]
df = pd.merge(df,filtered_df7,how='left',on='사업자번호')

#### 2-2 점수 계산

In [ ]:
#각 지표별 등급을 만들어준 컬럼의 타입을 categorical에서 float으로 변경해줌 -> 계산을 위해
df['영업이익율_등급'] = df['영업이익율_등급'].astype(float)
df['매출액평균_등급'] = df['매출액평균_등급'].astype(float)
df['매출액대비연구개발비_등급'] = df['매출액대비연구개발비_등급'].astype(float)
df['부채비율_등급'] = df['부채비율_등급'].astype(float)
df['기업규모_등급'] = df['기업규모_등급'].astype(float)
df['특허이노메인_등급'] = df['특허이노메인_등급'].astype(float)
df['수출여부_등급'] = df['수출여부_등급'].astype(float)
df['업종별수출비중_등급'] = df['업종별수출비중_등급'].astype(float)

In [ ]:
#등급의 값이 null이라는 것은 지표에 사용된 컬럼의 값이 null이라는 뜻으로 정보가 없다는 의미
#정보가 없으면 1점으로 만들어줌
columns_to_fill = ['영업이익율_등급', '매출액평균_등급','매출액대비연구개발비_등급', '부채비율_등급', '기업규모_등급', '특허이노메인_등급', '수출여부_등급', '업종별수출비중_등급']

df[columns_to_fill] = df[columns_to_fill].fillna(1)

In [ ]:
#점수 계산

#각 지표별로 점수와 가중치를 곱해 총 점수를 구함
df['점수'] = ((a/total) * df['영업이익율_등급']) + (h/total)*df['매출액평균_등급'] + ((b/total) * df['매출액대비연구개발비_등급']) + ((c/total)* df['부채비율_등급']) + ((d/total) * df['기업규모_등급']) + ((e/total) * df['특허이노메인_등급']) + ((f/total) * df['수출여부_등급'] + abs(g/total) * df['업종별수출비중_등급'])

##### 2-3 등급화
###### -> 1-5등급 + 정보없음

In [ ]:
#min값을 1로 설정하였을 때, 1값은 안들어가므로 min값은 0.9999999로 설정
min_value = 0.9999999
max_value = 10

# 5개의 구간으로 나누기 위해 6개의 경계값을 생성
bins = np.linspace(min_value, max_value, num=6)

# 생성된 경계값을 사용하여 데이터를 등급화(1-5등급으로 등급화)
df['등급'] = pd.cut(df['점수'], bins=bins, labels=[5, 4, 3, 2, 1], include_lowest=True)

##### 2-3-1 정보없음 등급 추가

In [ ]:
#아무 정보가 없어서 5등급이 나온 기업은 정보없음 등급으로 설정하기 위한 함수
def check_null(row):
  if(pd.isna(row['영업이익율']) & pd.isna(row['매출액평균']) & pd.isna(row['매출액대비연구개발비']) & pd.isna(row['부채비율구간']) & pd.isna(row['기업규모_수정'])
  & (row['특허이노메인_등급']==1) & (row['수출여부']==0) & (row['업종별수출비중_등급']==1)):
    return None
  else:
    return row['등급']

In [ ]:
df['등급'] = df.apply(check_null, axis=1)